In [1]:
import math

from lilgrad.core import Value
from lilgrad.utils import all_derivatives


In [2]:
def compare_results(l1, l2, acceptable_error):
    return [abs(v1 - v2) < acceptable_error for (v1, v2) in zip(l1, l2)], l1, l2


In [3]:
def check_grad(f, a, b=None, acceptable_error=1e-5):
    v1 = Value(a)

    if b is not None:
        v2 = Value(b)
        v3 = f(v1, v2)
    
        c_approx = all_derivatives(f, [a,b])
        v3.backward()
        c = [v1.grad, v2.grad]

    else:
        v2 = f(v1)
        c_approx = all_derivatives(f, [a])
        v2.backward()
        c = [v1.grad]
    
    return compare_results(c, c_approx, acceptable_error)
    

In [4]:
%psource Value.__mul__


    @input_to_value()
    @set_out_backward()
    def __mul__(self, other):
        out = Value(self.data * other.data, (self, other), _op='*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        
        return out, _backward


In [5]:
check_grad(lambda a, b: a * b, 50, 23)


([True, True], [23.0, 50.0], [23.00000005561742, 49.999999873762135])

In [6]:
%psource Value.__pow__


    @input_to_value()
    @set_out_backward()
    def __pow__(self, other):
        out = Value(self.data ** other.data, (self, other), _op='**')
        
        def _backward():
            self.grad += (other.data * (self.data ** (other.data - 1))) * out.grad
            other.grad += (out.data * math.log(self.data)) * out.grad

        return out, _backward


In [7]:
check_grad(lambda a, b: a**b, 2, 2)


([True, True],
 [4.0, 2.772588722239781],
 [3.9999999956741306, 2.7725887230545254])

In [8]:
%psource Value.exp


    @set_out_backward()
    def exp(self):
        out = Value(math.exp(self.data), (self,), _op='exp')

        def _backward():
            self.grad += (out.data) * out.grad

        return out, _backward


In [9]:
check_grad(lambda a: a.exp() if isinstance(a, Value) else math.exp(a), 3)


([True], [20.085536923187668], [20.085536931446768])

In [10]:
%psource Value.__truediv__


    def __truediv__(self, other):
        return self * (other ** -1)


In [11]:
check_grad(lambda a, b: a / b, 289, 49)


([True, True],
 [0.02040816326530612, -0.1203665139525198],
 [0.020408159606688514, -0.12036651675373378])

In [12]:
%psource Value.log


    @set_out_backward()
    def log(self):
        out = Value(math.log(self.data), (self,), _op='log')

        def _backward():
            self.grad += (1 / self.data) * out.grad

        return out, _backward


In [13]:
check_grad(lambda a: a.log() if isinstance(a, Value) else math.log(a), 3)


([True], [0.3333333333333333], [0.3333333331578814])

In [14]:
%psource Value.sin


    @set_out_backward()
    def sin(self):
        out = Value(math.sin(self.data), (self,), _op='sin')
        
        def _backward():
            self.grad += (math.cos(self.data)) * out.grad
        
        return out, _backward


In [15]:
check_grad(lambda a: a.sin() if isinstance(a, Value) else math.sin(a), 3)


([True], [-0.9899924966004454], [-0.9899924972855967])

In [16]:
%psource Value.cos


    @set_out_backward()
    def cos(self):
        out = Value(math.cos(self.data), (self,), _op='cos')
        
        def _backward():
            self.grad += (-math.sin(self.data)) * out.grad
        
        return out, _backward


In [17]:
check_grad(lambda a: a.cos() if isinstance(a, Value) else math.cos(a), 3)


([True], [-0.1411200080598672], [-0.14112000845667438])

In [18]:
%psource Value.tan


    @set_out_backward()
    def tan(self):
        out = Value(math.tan(self.data), (self,), _op='tan')

        def _backward():
            self.grad += (1 / (math.cos(self.data) ** 2)) * out.grad
        
        return out, _backward


In [19]:
check_grad(lambda a: a.tan() if isinstance(a, Value) else math.tan(a), 3)


([True], [1.020319516942427], [1.0203195176250457])

In [20]:
%psource Value.sinh


    @set_out_backward()
    def sinh(self):
        out = Value(math.sinh(self.data), (self,), _op='sinh')

        def _backward():
            self.grad += (math.cosh(self.data)) * out.grad
        
        return out, _backward


In [21]:
check_grad(lambda a: a.sinh() if isinstance(a, Value) else math.sinh(a), 3)


([True], [10.067661995777765], [10.067662001844724])

In [22]:
%psource Value.cosh


    @set_out_backward()
    def cosh(self):
        out = Value(math.cosh(self.data), (self,), _op='cosh')

        def _backward():
            self.grad += (math.sinh(self.data)) * out.grad
        
        return out, _backward


In [23]:
check_grad(lambda a: a.cosh() if isinstance(a, Value) else math.cosh(a), 3)


([True], [10.017874927409903], [10.017874929602044])

In [24]:
%psource Value.tanh


    @set_out_backward()
    def tanh(self):
        out = Value(math.tanh(self.data), (self,), _op='tanh')

        def _backward():
            self.grad += (1 / (math.cosh(self.data) ** 2)) * out.grad
        
        return out, _backward


In [25]:
check_grad(lambda a: a.tanh() if isinstance(a, Value) else math.tanh(a), 3)


([True], [0.009866037165440192], [0.00986603687636034])

In [26]:
%psource Value.sigmoid


    def sigmoid(self):
        return 1 / (1 + (-self).exp())


In [ ]:
def sigmoid(x):
    1 / (1 + math.exp(-x))


In [27]:
check_grad(lambda a: a.sigmoid() if isinstance(a, Value) else x, 3)


([True], [0.045176659730912144], [0.045176660190549])

In [28]:
%psource Value.relu


    @set_out_backward()
    def relu(self):
        out = Value(max(0, self.data), (self,), _op='relu')
        
        def _backward():
            self.grad += out.grad if out.data > 0 else 0

        return out, _backward


In [29]:
check_grad(lambda a: a.relu() if isinstance(a, Value) else max(a, 0), 3)


([True], [1.0], [1.0000000005838672])

In [30]:
def more_complicated_fn(a, b):
    return 3 + a.pow(b).sigmoid().relu().exp().sin()


check_grad(lambda a, b: more_complicated_fn(a, b))
